In [1]:
# import re
# import io
# import requests


# from google.oauth2.credentials import Credentials
# from googleapiclient.discovery import build
# from googleapiclient.errors import HttpError
# from pdfminer.converter import TextConverter
# from pdfminer.pdfinterp import PDFPageInterpreter
# from pdfminer.pdfinterp import PDFResourceManager
# from pdfminer.pdfpage import PDFPage

In [2]:
from bs4 import BeautifulSoup
import re
import requests

In [3]:
response = requests.get(
    url='https://proxy.scrapeops.io/v1/',
    params={
        'api_key': '378d2b00-d774-4869-bf97-325ac3d1de22',
        'url': 'https://www.al-akhbar.com/Editions',
    },
    )
soup = BeautifulSoup(response.text, 'html.parser')

In [10]:
container = soup.find('div', {'class': 'l-content row archive-issues'})
container = container.find('div', id='archive-days-wrap')

In [17]:
container.findChildren('a')

[<a class="day" data-hasqtip="0" data-id="4816" data-name="20230104" data-oldtitle="العدد 4816" href="/Editions/2023/1/4" title="">
                                                     4
                                                 </a>,
 <a class="day" data-hasqtip="0" data-id="4817" data-name="20230105" data-oldtitle="العدد 4817" href="/Editions/2023/1/5" title="">
                                                     5
                                                 </a>,
 <a class="day" data-hasqtip="0" data-id="4818" data-name="20230106" data-oldtitle="العدد 4818" href="/Editions/2023/1/6" title="">
                                                     6
                                                 </a>,
 <a class="day" data-hasqtip="0" data-id="4819" data-name="20230109" data-oldtitle="العدد 4819" href="/Editions/2023/1/9" title="">
                                                     9
                                                 </a>,
 <a class="day" data-hasqtip="0"

In [12]:
count = 0
# for link in container.find_all('a',
#                           attrs={'href': re.compile('^/Editions/')}):
    # display the actual urls
    
    # count +=1
    # if count >= 5:
    #     print(link.find_parent('div'))
    # elif count == 7:
    #     break
    # else:
    #     continue

In [5]:
# url = 'https://www.al-akhbar.com/Editions'
# response = requests.get(url)
# soup = BeautifulSoup(response.text, 'html.parser')
# body = soup.findChildren()
# pdf_links = soup.find_all('a', href=lambda x: x and x.startswith('/Editions'))
# def save_article_to_drive(article_text, file_name):try:
#       # authenticate with Google Drive
#       credentials = Credentials.from_authorized_user_info()
#       service = build('drive', 'v3', credentials=credentials)

#       # create the file on Google Drive
#       file_metadata = {'name': file_name, 'mimeType': 'text/plain'}
#       media = MediaIoBaseUpload(io.StringIO(article_text), mimetype='text/plain',chunksize=1024*1024, resumable=True)
#       file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
#       print(f'File ID: {file.get(\id\)}')
#   except HttpError as error:
#       print(f'An error occurred: {error}')
#       file = None
#   return file"

# "def extract_text_from_pdf(pdf_content):
# "  resource_manager = PDFResourceManager()
# "  fake_file_handle = io.StringIO()
# "  converter = TextConverter(resource_manager, fake_file_handle)
# "  page_interpreter = PDFPageInterpreter(resource_manager, converter)
# "
# "  with io.BytesIO(pdf_content) as fh:
# "    for page in PDFPage.get_pages(fh,
# "                                  caching=True,
# "                                  check_extractable=True):
# "      page_interpreter.process_page(page)
# "
# "    text = fake_file_handle.getvalue()
# "
# "  # close open handles
# "  converter.close()
# "  fake_file_handle.close()
# "
# "  return text"
 
# "# scrape the website for PDF links
# "
# "
# "
# "# # download each PDF and save it to Google Drive
# "# text_ls = [] 
# "# for link in pdf_links:
# "#   # download the PDF
# "#   pdf_url = link['href']
# "#   pdf_response = requests.get(pdf_url)
# "#   pdf_content = pdf_response.content
# "
# "#   # extract the text from the PDF
# "#   text = extract_text_from_pdf(pdf_content)
# "#   text_ls.append(text)
# "print(soup)"
  
# "# search the text for keywords
# "keywords = ['keyword1', 'keyword2', 'keyword3']
# "for keyword in keywords:
# "  if re.search(keyword, text, re.IGNORECASE):
# "    # keyword found, save the article
# "    file_name = f'{keyword}_article.txt'
# "    save_article_to_drive(text, file_name)
# "    break"